In [2]:
from Cryptodome.Protocol.KDF import PBKDF2
from Crypto.Hash import SHA512
from Crypto.Util.number import bytes_to_long, long_to_bytes,getPrime, inverse
import Crypto.Random.random as random
from Cryptodome.Cipher import AES

In [3]:
global g,p
g=2
p=65537
def DH_gen():
    a=random.randint(100,1000)
    return (a,pow(g,a,p))
def EG_sign(m,x,y):
    k=getPrime(12)
    r=pow(g,k,p)
    s=(m-x*r)*inverse(k,p-1)%(p-1)
    return(r,s)
def EG_check(y,r,s,m):
    return(pow(g,m,p)==pow(y,r,p)*pow(r,s,p)%p)

In [4]:
class server:
    #инициализация сервера
    def __init__(self):
        self.users={}
        self.chats={}
        self.clients={}
        pass
    #регистрация пользователей
    def registr(self,client, pik, ppk, ppk_sign, pok):
        self.users[client.name]={'name':client.name, 'pik':pik, 'ppk':ppk, 'ppk_sign':ppk_sign, 'pok':pok}
        self.clients[client.name]=client
        self.chats[client.name]={}
    #предоставление данных о клиентах 
    def keys_share(self, name):
        bpik=self.users[name]['pik']
        bppk=self.users[name]['ppk']
        bppk_sign=self.users[name]['ppk_sign']
        bpok=self.users[name]['pok']
        return(bpik, bppk , bppk_sign, bpok)
    #def key_upd(self,client, )

In [5]:
class client:
    #инициализация клиентов
    def __init__(self,name,server):
        self.server=server
        self.name=name
        self.chats={}
    #регситрация клиентов
    def registr(self):
        sik,pik=DH_gen()
        spk,ppk=DH_gen()
        ppk_sign=EG_sign(ppk,sik,pik)
        sok, pok=DH_gen()
        self.sik=sik
        self.spk=spk
        self.sok=sok
        self.server.registr(self, pik, ppk, ppk_sign, pok)
    #инициализация общения
    def start_conversation(self,partner_name):
        self.chats[partner_name]={}
        bpik, bppk, bppk_sign, bpok = self.server.keys_share(partner_name)
        if not (EG_check(bpik,bppk_sign[0],bppk_sign[1], bppk)):
            print("Ключ был подменен")
            return
        DH1=long_to_bytes(pow(bppk,self.sik,p), 16)
        sek, pek= DH_gen()
        DH2=long_to_bytes(pow(bpik,sek,p), 16)
        DH3=long_to_bytes(pow(bppk,sek,p), 16)
        if(bpok==False):
            SK=PBKDF2(DH1+DH2+DH3, b'', 16, count=100, hmac_hash_module=SHA512)
            self.chats[partner_name]={'RK':SK}
            return
        DH4=long_to_bytes(pow(bpok,sek,p), 16)
        SK=PBKDF2(DH1+DH2+DH3+DH4, b'', 16, count=100, hmac_hash_module=SHA512)
        self.chats[partner_name]['RK']=SK
        self.chats[partner_name]['dir']=1
        self.chats[partner_name]['n']=1
        sk,pk=DH_gen()
        self.chats[partner_name]['keys']={'sk':[SK],'ak':sk}
        self.chats[partner_name]['keys']['RK']=SK
        cipher = AES.new(SK, AES.MODE_GCM, nonce=b'a')
        
        message={'name':self.name,'ik':pow(g,self.sik,p), 'pek':pek, 'id':-1, 'msg':cipher.encrypt(b'Hello!')}
        self.server.chats[self.name][partner_name]={'msgs':[message]}
        self.server.chats[partner_name][self.name]={'msgs':[message]}
        self.server.chats[self.name][partner_name]['keys']=pk
        self.server.chats[self.name][partner_name]['n']=1
    #обновление диалогов   
    def update_dialog(self,name):
        try:
            self.chats[name]['msgs']=self.server.chats[self.name][name]['msgs'].copy()
        except:
            self.chats[name]={}
            self.chats[name]['msgs']=self.server.chats[self.name][name]['msgs'].copy()
    #проверка наличия клиентов, желаающих начать общение   
    def init_check(self):
        for name in self.chats:
            if(len(self.chats[name]['msgs'])==1):
                init_message=self.chats[name]['msgs'][0]
                name=init_message['name']
                ik=init_message['ik']
                pek=init_message['pek']
                msg=init_message['msg']
                DH1=long_to_bytes(pow(ik,self.spk,p), 16)
                DH2=long_to_bytes(pow(pek,self.sik,p), 16)
                DH3=long_to_bytes(pow(pek,self.spk,p), 16)
                if(self.sok==False):
                    SK=PBKDF2(DH1+DH2+DH3, b'', 16, count=100, hmac_hash_module=SHA512)
                    self.chats[name]['RK']=SK
                    return
                DH4=long_to_bytes(pow(pek,self.sok,p), 16)  
                SK=PBKDF2(DH1+DH2+DH3+DH4, b'', 16, count=100, hmac_hash_module=SHA512)
                self.chats[name]['dir']=1
                self.chats[name]['n']=1
                sk,pk=DH_gen()
                self.chats[name]['keys']={'sk':[SK],'ak':sk}
                self.chats[name]['keys']['RK']=SK
                self.server.chats[self.name][name]['keys']=pk
                self.server.chats[self.name][name]['n']=1
    #отправка сообщений            
    def msg_send(self,name,text):
        if(self.chats[name]['dir']==1):
            sk,pk=DH_gen()
            self.chats[name]['keys']['ak']=sk
            bpk=self.server.chats[name][self.name]['keys']
            self.server.chats[self.name][name]['keys']=pk
            dhKEY=long_to_bytes(pow(bpk,sk,p),16)
            symk=PBKDF2(dhKEY, self.chats[name]['keys']['RK'], 16, count=100, hmac_hash_module=SHA512)
            self.chats[name]['keys']['sk'].append(symk)
            self.chats[name]['keys']['RK']=symk
            message=text.encode('utf-8')
            cipher = AES.new(symk, AES.MODE_GCM, nonce=b'a')
            cmsg=cipher.encrypt(message)
            self.chats[name]['msgs'].append(text)
            self.server.chats[self.name][name]['msgs'].append(cmsg)
            self.server.chats[name][self.name]['msgs'].append(cmsg)
            self.server.chats[self.name][name]['n']=self.server.chats[self.name][name]['n']+1
            self.server.chats[name][self.name]['n']=self.server.chats[name][self.name]['n']+1
            self.chats[name]['dir']=0
            self.chats[name]['n']=self.chats[name]['n']+1
        else:
            print('2')
            symk=PBKDF2(self.chats[name]['keys']['sk'][-1], self.chats[name]['keys']['RK'], 16, count=100, hmac_hash_module=SHA512)
            self.chats[name]['keys']['sk'].append(symk)
            self.chats[name]['keys']['RK']=symk
            message=text.encode('utf-8')
            self.chats[name]['msgs'].append(text)
            cipher = AES.new(symk, AES.MODE_GCM, nonce=b'a')
            cmsg=cipher.encrypt(message)
            self.server.chats[self.name][name]['msgs'].append(cmsg)
            self.server.chats[name][self.name]['msgs'].append(cmsg)
            self.server.chats[self.name][name]['n']=self.server.chats[self.name][name]['n']+1
            self.server.chats[name][self.name]['n']=self.server.chats[name][self.name]['n']+1
            self.chats[name]['n']=self.chats[name]['n']+1
      #проверка входящих сообщений        
    def msg_check(self,name):
        msg_len=len(self.server.chats[self.name][name]['msgs'])
        if(msg_len>self.chats[name]['n']):
            msgs_count=msg_len-self.chats[name]['n']
            bpk=self.server.chats[name][self.name]['keys']
            dhKEY=long_to_bytes(pow(bpk,self.chats[name]['keys']['ak'],p),16)
            symk=PBKDF2(dhKEY, self.chats[name]['keys']['RK'], 16, count=100, hmac_hash_module=SHA512)
            self.chats[name]['keys']['sk'].append(symk)
            self.chats[name]['keys']['RK']=symk
            new_msg=self.server.chats[self.name][name]['msgs'][self.chats[name]['n']]
            cipher = AES.new(self.chats[name]['keys']['sk'][-1], AES.MODE_GCM, nonce=b'a')
            text=cipher.decrypt(new_msg).decode('utf-8')
            print(text)
            self.chats[name]['n']=self.chats[name]['n']+1
            self.chats[name]['msgs'].append(text)
            for i in range(1,msgs_count):
                symk=PBKDF2(self.chats[name]['keys']['sk'][-1], self.chats[name]['keys']['RK'], 16, count=100, hmac_hash_module=SHA512)
                self.chats[name]['keys']['sk'].append(symk)
                new_msg=self.server.chats[self.name][name]['msgs'][self.chats[name]['n']+i]
                cipher = AES.new(self.chats[name]['keys']['sk'][-1], AES.MODE_GCM, nonce=b'a')
                text=cipher.decrypt(new_msg).decode('utf-8')
                print(text)
                self.chats[name]['msgs'].append(text)
                self.chats[name]['n']=self.chats[name]['n']+1
            self.chats[name]['dir']=1

In [6]:
s=server()

In [7]:
c1=client('c1',s)
c1.registr()

In [8]:
c2=client('c2',s)
c2.registr()

In [9]:
s.clients

{'c1': <__main__.client at 0x1da8276acc8>,
 'c2': <__main__.client at 0x1da827703c8>}

In [10]:
c1.start_conversation('c2')

In [11]:
c1.update_dialog('c2')

In [12]:
c2.update_dialog('c1')

In [13]:
c2.init_check()

In [14]:
s.chats

{'c1': {'c2': {'msgs': [{'name': 'c1',
     'ik': 256,
     'pek': 65281,
     'id': -1,
     'msg': b'Ib\xde\x8b#_'}],
   'keys': 1024,
   'n': 1}},
 'c2': {'c1': {'msgs': [{'name': 'c1',
     'ik': 256,
     'pek': 65281,
     'id': -1,
     'msg': b'Ib\xde\x8b#_'}],
   'keys': 32,
   'n': 1}}}

In [15]:
c1.msg_send('c2','aaaaaaaaaaaaa')

In [16]:
c1.chats

{'c2': {'RK': b'n\x8a|\x87(KM*\xee\xb9\xf3n\xb3W\x92]',
  'dir': 0,
  'n': 2,
  'keys': {'sk': [b'n\x8a|\x87(KM*\xee\xb9\xf3n\xb3W\x92]',
    b'\xac\xbc<]Y\x94\x8a\xa2\x13\x18\x82{\t#7x'],
   'ak': 515,
   'RK': b'\xac\xbc<]Y\x94\x8a\xa2\x13\x18\x82{\t#7x'},
  'msgs': [{'name': 'c1',
    'ik': 256,
    'pek': 65281,
    'id': -1,
    'msg': b'Ib\xde\x8b#_'},
   'aaaaaaaaaaaaa']}}

In [17]:
c2.msg_check('c1')

aaaaaaaaaaaaa


In [18]:
c2.chats

{'c1': {'msgs': [{'name': 'c1',
    'ik': 256,
    'pek': 65281,
    'id': -1,
    'msg': b'Ib\xde\x8b#_'},
   'aaaaaaaaaaaaa'],
  'dir': 1,
  'n': 2,
  'keys': {'sk': [b'n\x8a|\x87(KM*\xee\xb9\xf3n\xb3W\x92]',
    b'\xac\xbc<]Y\x94\x8a\xa2\x13\x18\x82{\t#7x'],
   'ak': 677,
   'RK': b'\xac\xbc<]Y\x94\x8a\xa2\x13\x18\x82{\t#7x'}}}

In [19]:
c1.chats

{'c2': {'RK': b'n\x8a|\x87(KM*\xee\xb9\xf3n\xb3W\x92]',
  'dir': 0,
  'n': 2,
  'keys': {'sk': [b'n\x8a|\x87(KM*\xee\xb9\xf3n\xb3W\x92]',
    b'\xac\xbc<]Y\x94\x8a\xa2\x13\x18\x82{\t#7x'],
   'ak': 515,
   'RK': b'\xac\xbc<]Y\x94\x8a\xa2\x13\x18\x82{\t#7x'},
  'msgs': [{'name': 'c1',
    'ik': 256,
    'pek': 65281,
    'id': -1,
    'msg': b'Ib\xde\x8b#_'},
   'aaaaaaaaaaaaa']}}

In [20]:
s.chats

{'c1': {'c2': {'msgs': [{'name': 'c1',
     'ik': 256,
     'pek': 65281,
     'id': -1,
     'msg': b'Ib\xde\x8b#_'},
    b'H\xb5\xdd\x13\x92)%1os\x85\xbf1'],
   'keys': 8,
   'n': 2}},
 'c2': {'c1': {'msgs': [{'name': 'c1',
     'ik': 256,
     'pek': 65281,
     'id': -1,
     'msg': b'Ib\xde\x8b#_'},
    b'H\xb5\xdd\x13\x92)%1os\x85\xbf1'],
   'keys': 32,
   'n': 2}}}

In [21]:
c2.msg_send('c1','aaaaaaaaaaaaa')

In [22]:
c1.msg_check('c2')

aaaaaaaaaaaaa


In [23]:
c1.chats

{'c2': {'RK': b'n\x8a|\x87(KM*\xee\xb9\xf3n\xb3W\x92]',
  'dir': 1,
  'n': 3,
  'keys': {'sk': [b'n\x8a|\x87(KM*\xee\xb9\xf3n\xb3W\x92]',
    b'\xac\xbc<]Y\x94\x8a\xa2\x13\x18\x82{\t#7x',
    b'XO\xa8T\x85+FE\x81\x8f%\x16&\x90\xee\xc0'],
   'ak': 515,
   'RK': b'XO\xa8T\x85+FE\x81\x8f%\x16&\x90\xee\xc0'},
  'msgs': [{'name': 'c1',
    'ik': 256,
    'pek': 65281,
    'id': -1,
    'msg': b'Ib\xde\x8b#_'},
   'aaaaaaaaaaaaa',
   'aaaaaaaaaaaaa']}}

In [24]:
c2.chats

{'c1': {'msgs': [{'name': 'c1',
    'ik': 256,
    'pek': 65281,
    'id': -1,
    'msg': b'Ib\xde\x8b#_'},
   'aaaaaaaaaaaaa',
   'aaaaaaaaaaaaa'],
  'dir': 0,
  'n': 3,
  'keys': {'sk': [b'n\x8a|\x87(KM*\xee\xb9\xf3n\xb3W\x92]',
    b'\xac\xbc<]Y\x94\x8a\xa2\x13\x18\x82{\t#7x',
    b'XO\xa8T\x85+FE\x81\x8f%\x16&\x90\xee\xc0'],
   'ak': 361,
   'RK': b'XO\xa8T\x85+FE\x81\x8f%\x16&\x90\xee\xc0'}}}

In [25]:
s.chats

{'c1': {'c2': {'msgs': [{'name': 'c1',
     'ik': 256,
     'pek': 65281,
     'id': -1,
     'msg': b'Ib\xde\x8b#_'},
    b'H\xb5\xdd\x13\x92)%1os\x85\xbf1',
    b'"\x9a\xd9\x02\x1fDmT\xad\x91\x99\x9bn'],
   'keys': 8,
   'n': 3}},
 'c2': {'c1': {'msgs': [{'name': 'c1',
     'ik': 256,
     'pek': 65281,
     'id': -1,
     'msg': b'Ib\xde\x8b#_'},
    b'H\xb5\xdd\x13\x92)%1os\x85\xbf1',
    b'"\x9a\xd9\x02\x1fDmT\xad\x91\x99\x9bn'],
   'keys': 512,
   'n': 3}}}

In [34]:
a

{'g': {'dsd': 'ddd'}}

In [35]:
a={'dd':'1', 'qe':'4'}